# Lab 2: Analyzing Water Networks and Energy Use in CA

In this lab, we will be visualizing and analyzing water networks and specific utilities in CA.

In [2]:
from sub_network import subWESTnet
import pandas as pd
import random

Here we will randomize which utility you will be visualizing.

In [3]:
utility_dict = {0:'1803033E', 1:'1805003E', 2:'1809027E', 3:'1810019E', 4:'1805085E', 5:'1807341E'}
my_utility = utility_dict[random.randint(0, 5)]
print(my_utility)

1805085E


Here, we create two graph instances, one upstream from the utility and one downstream from a source that utility is associated with. (I haven't implemented the randomized utility yet, so it's just using the 1805003E utility right now)

In [15]:
upstream_example = subWESTnet('data/links_erl.csv', 2010)
downstream_example = subWESTnet('data/links_erl.csv', 2010)

In [16]:
upstream_example.upstream('1805003E')

Unweighted upstream graph built.


In [17]:
upstream_example.balance_graph()

Graph is weighted.


In [18]:
downstream_example.downstream('SW1805SWP08')
downstream_example.balance_graph()

Unweighted downstream graph complete.
Graph is weighted.


The following attributes of our upstream and downstream examples show the tables of the actually network (by using source and target labels) and the energies associated with each node in the graph.

In [19]:
upstream_example.table()

,source,target,cumulative_volume_af,transmission_kwh/af,treatment_kwh/af,used_vol_af
0,SW1805SWP08,GWR1805-2-09.01,2.490000e+04,83.9285,442.626506,0.000000
1,GWR1805-2-09.01,1805003E,1.570000e+04,318.2000,0.000000,0.000000
2,GW1805-2-09.01,1805003PD,2.730722e+04,352.0000,181.750000,27307.216490
3,GW1805-2-09,GW1805-2-09.01,2.730722e+04,0.0000,0.000000,27307.216490
4,RES_DELVALL,1805003PD,3.191753e+03,87.0000,224.000000,3191.752577
5,SW1805081EB,1805003PD,1.037320e+04,0.0000,26.000000,10373.195880
6,RES1805081,SW1805081SYS,7.687000e+03,0.0000,0.000000,0.000000
7,R_Tuolumn,RES1805081,1.000000e+04,0.0000,0.000000,0.000000
8,SW1805081SYS,SW1805081EB,1.146900e+04,0.7000,0.000000,0.000000
9,SW1805SWP08,1805003PD,2.127835e+03,83.9285,224.000000,2127.835052


In [9]:
downstream_example.table()

,source,target,cumulative_volume_af,transmission_kwh/af,treatment_kwh/af,used_vol_af
0,SW1805SWP08,SWP_YBACRD,104.022488,83.9285,224.000000,0.000000
1,SW1805SWP08,1805003PD,2127.835052,83.9285,224.000000,2127.835052
2,1805003PD,1805003E,43000.000000,318.2000,0.000000,43000.000000
3,SW1805SWP08,GWR1805-2-09.01,24900.000000,83.9285,442.626506,0.000000
4,GWR1805-2-09.01,1805001PD,9200.000000,352.0000,26.000000,0.000000
5,1805001PD,1805047PD,3897.492840,25.0000,0.000000,3890.864033
6,1805047PD,1805047E,3890.864033,318.2000,0.000000,3890.864033
7,1805001PD,1805013PD,6849.273294,25.0000,0.000000,6803.900000
8,1805013PD,1805013E,9747.000000,300.0000,0.000000,9747.000000
9,1805001PD,1805029PD,8801.000000,25.0000,0.000000,8742.000000


In [10]:
upstream_example.energy

,node,kwh/af
0,Lk_OrovilleSWP,0.000000
1,GW1805-2-09,0.000000
2,RES_DELVALL,0.000000
3,R_Tuolumn,0.000000
4,R_Calaveras,0.000000
5,R_Mokelumn,0.000000
6,R_Stanislaus,0.000000
7,R_SanJoaquin,0.000000
8,LK_ShastaCVP,0.000000
9,LK_TrinityCVP,0.000000


In [11]:
downstream_example.energy

,node,kwh/af
0,SW1805SWP08,0.000000
1,SWP_YBACRD,307.928500
2,1805003PD,307.928500
3,GWR1805-2-09.01,526.555006
4,1805003E,684.602710
5,1805001PD,427.061690
6,1805047PD,452.061690
7,1805047E,770.261690
8,1805013PD,452.061690
9,1805013E,752.061690


These two cells, take the downstream and upstream tables, and turn them into a dot file. The dotfile is placed in 

ouput/sub_(your_utility_code)_2010__upstream/downstream.dot

In [12]:
upstream_example.to_dot()

In [13]:
downstream_example.to_dot()

go to the dot file (inside the ouput directory where the lab2 notebook is located) and paste the code in the dot file [here](https://dreampuf.github.io/GraphvizOnline/). Make sure to replace the entire code block that is already in the link with the entire code block in our output.dot files.